In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# It's good practice to first uninstall potentially conflicting packages
!pip uninstall torch torchvision torchaudio transformers accelerate bitsandbytes torchao -y

# Install PyTorch (ensure compatibility with CUDA version on Kaggle GPU)
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

# Install OpenMIM
!pip install -U openmim

# Install MMEngine
!mim install mmengine

# Install a compatible version of MMCV as per the previous error message
!pip uninstall mmcv -y # Uninstall current mmcv first to be sure
!mim install "mmcv>=2.0.0rc4,<2.2.0" 

# Install specific, potentially older but more stable, versions of transformers and accelerate
# These versions are chosen to reduce the likelihood of issues with very new torchao features.
!pip install transformers==4.30.2 
!pip install accelerate==0.22.0 

# Now install mmdet and mmpose. These should pick up the already installed compatible libraries.
!mim install "mmdet>=3.0.0" 
!mim install "mmpose>=1.0.0"

# Install other necessary libraries
!pip install opencv-python numpy tqdm pandas openpyxl requests

In [ ]:
import torch
import torchvision
import mmpose
import mmdet
import mmcv
import mmengine # Ensure this is imported
import cv2
import numpy as np
import os
import pandas as pd
import requests
import subprocess
import re
from pathlib import Path
from tqdm import tqdm
# import tempfile # Not strictly needed now

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

print(f"MMPose version: {mmpose.__version__}")
print(f"MMDetection version: {mmdet.__version__}")
print(f"MMCV version: {mmcv.__version__}")
print(f"MMEngine version: {mmengine.__version__}") # <<< ADD THIS LINE
print(f"Pandas version: {pd.__version__}")
print(f"Requests version: {requests.__version__}")
print(f"OpenCV version: {cv2.__version__}")

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(f"MMCV CUDA version: {get_compiling_cuda_version()}")
print(f"MMCV compiler version: {get_compiler_version()}")

ffmpeg_check = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
if ffmpeg_check.returncode == 0:
    print("ffmpeg found.")
else:
    print("ffmpeg not found. Segmentation will fail.")

In [ ]:
import os
import requests
import torch # For torch.hub.download_url_to_file
import subprocess # For git clone

# --- Directories ---
BASE_WORKING_DIR = '/kaggle/working/'
CHECKPOINTS_DIR = os.path.join(BASE_WORKING_DIR, 'checkpoints')
MMDET_DIR = os.path.join(BASE_WORKING_DIR, 'mmdetection')
MMPOSE_DIR = os.path.join(BASE_WORKING_DIR, 'mmpose')

os.makedirs(CHECKPOINTS_DIR, exist_ok=True)

# --- Clone MMDetection and MMPose Repositories for Config Files ---
# We'll clone specific versions/tags if known, otherwise main branch.
# For mmdet>=3.0.0, let's try to get a recent stable tag or main.
# For mmpose>=1.0.0, similar approach.
MMDET_REPO_URL = "https://github.com/open-mmlab/mmdetection.git"
MMPOSE_REPO_URL = "https://github.com/open-mmlab/mmpose.git"
MMDET_TAG = "v3.1.0" # A recent stable tag for mmdet 3.x
MMPOSE_TAG = "v1.1.0" # A recent stable tag for mmpose 1.x


def clone_repo_if_not_exists(repo_url, target_dir, tag=None):
    if not os.path.exists(os.path.join(target_dir, '.git')): # Check if it's a git repo
        print(f"Cloning {repo_url} (tag: {tag if tag else 'latest'}) to {target_dir}...")
        clone_command = ['git', 'clone']
        if tag:
            clone_command.extend(['-b', tag])
        clone_command.extend([repo_url, target_dir])
        
        try:
            subprocess.run(clone_command, check=True, capture_output=True, text=True)
            print(f"Successfully cloned {repo_url} to {target_dir}")
        except subprocess.CalledProcessError as e:
            print(f"Error cloning {repo_url}:")
            print(f"Command: {' '.join(e.cmd)}")
            print(f"Return code: {e.returncode}")
            print(f"Stdout: {e.stdout}")
            print(f"Stderr: {e.stderr}")
            raise # Re-raise the exception to stop execution if cloning fails
    else:
        print(f"Repository already exists at {target_dir}, skipping clone.")

clone_repo_if_not_exists(MMDET_REPO_URL, MMDET_DIR, MMDET_TAG)
clone_repo_if_not_exists(MMPOSE_REPO_URL, MMPOSE_DIR, MMPOSE_TAG)


# --- Pose Estimation Model (RTMPose-L Wholebody) ---
# Config path now points within the cloned mmpose repo
local_pose_config_file = os.path.join(MMPOSE_DIR, 'configs/wholebody_2d_keypoint/rtmpose/coco-wholebody/rtmpose-l_8xb32-270e_coco-wholebody-384x288.py')
pose_checkpoint_file_url = 'https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-l_simcc-coco-wholebody_pt-aic-coco_270e-384x288-eaeb96c8_20230125.pth'
local_pose_checkpoint_file = os.path.join(CHECKPOINTS_DIR, 'rtmpose-l_coco-wholebody.pth')

# --- Object Detection Model (Faster R-CNN R50 FPN) ---
# Config path now points within the cloned mmdetection repo
local_det_config_file = os.path.join(MMDET_DIR, 'configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')
det_checkpoint_file_url = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
local_det_checkpoint_file = os.path.join(CHECKPOINTS_DIR, 'faster_rcnn_r50_fpn_1x_coco.pth')


def download_checkpoint_if_not_exists(url, local_path):
    if not os.path.exists(local_path):
        print(f"Downloading checkpoint {url} to {local_path}...")
        try:
            # Using torch.hub.download_url_to_file for checkpoints
            torch.hub.download_url_to_file(url, local_path, progress=True)
            print("Download complete.")
        except Exception as e:
            print(f"Error downloading checkpoint {url}: {e}")
            if os.path.exists(local_path): # Clean up partial download
                os.remove(local_path)
            raise
    else:
        print(f"Checkpoint {local_path} already exists.")

download_checkpoint_if_not_exists(pose_checkpoint_file_url, local_pose_checkpoint_file)
download_checkpoint_if_not_exists(det_checkpoint_file_url, local_det_checkpoint_file)

# Verify that config files exist at their new paths
if not os.path.exists(local_pose_config_file):
    print(f"ERROR: Pose config file not found: {local_pose_config_file}")
    print("Please check the mmpose repository structure and path.")
else:
    print(f"Pose config file found: {local_pose_config_file}")

if not os.path.exists(local_det_config_file):
    print(f"ERROR: Detection config file not found: {local_det_config_file}")
    print("Please check the mmdetection repository structure and path.")
else:
    print(f"Detection config file found: {local_det_config_file}")
    
print("\nModel configuration paths updated to use cloned repos. Checkpoint paths set.")

In [ ]:
from mmdet.apis import init_detector, inference_detector
from mmpose.apis import init_model as init_pose_estimator
from mmengine.registry import DefaultScope # Import DefaultScope class for its static methods

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# --- Manually Define COCO-WholeBody Dataset Info (133 keypoints) ---
# (COCO_WHOLEBODY_KEYPOINT_NAMES and OFFICIAL_MMPOSE_COCO_WHOLEBODY_SKELETON_LINKS definitions remain here as before)
COCO_WHOLEBODY_KEYPOINT_NAMES = [
    'kpt_0', 'kpt_1', 'kpt_2', 'kpt_3', 'kpt_4', 'kpt_5', 'kpt_6', 'kpt_7', 'kpt_8', 'kpt_9', 
    'kpt_10', 'kpt_11', 'kpt_12', 'kpt_13', 'kpt_14', 'kpt_15', 'kpt_16', 'kpt_17', 'kpt_18', 'kpt_19',
    'kpt_20', 'kpt_21', 'kpt_22', 'kpt_23', 'kpt_24', 'kpt_25', 'kpt_26', 'kpt_27', 'kpt_28', 'kpt_29',
    'kpt_30', 'kpt_31', 'kpt_32', 'kpt_33', 'kpt_34', 'kpt_35', 'kpt_36', 'kpt_37', 'kpt_38', 'kpt_39',
    'kpt_40', 'kpt_41', 'kpt_42', 'kpt_43', 'kpt_44', 'kpt_45', 'kpt_46', 'kpt_47', 'kpt_48', 'kpt_49',
    'kpt_50', 'kpt_51', 'kpt_52', 'kpt_53', 'kpt_54', 'kpt_55', 'kpt_56', 'kpt_57', 'kpt_58', 'kpt_59',
    'kpt_60', 'kpt_61', 'kpt_62', 'kpt_63', 'kpt_64', 'kpt_65', 'kpt_66', 'kpt_67', 'kpt_68', 'kpt_69',
    'kpt_70', 'kpt_71', 'kpt_72', 'kpt_73', 'kpt_74', 'kpt_75', 'kpt_76', 'kpt_77', 'kpt_78', 'kpt_79',
    'kpt_80', 'kpt_81', 'kpt_82', 'kpt_83', 'kpt_84', 'kpt_85', 'kpt_86', 'kpt_87', 'kpt_88', 'kpt_89',
    'kpt_90', 'kpt_91', 'kpt_92', 'kpt_93', 'kpt_94', 'kpt_95', 'kpt_96', 'kpt_97', 'kpt_98', 'kpt_99',
    'kpt_100', 'kpt_101', 'kpt_102', 'kpt_103', 'kpt_104', 'kpt_105', 'kpt_106', 'kpt_107', 'kpt_108', 'kpt_109',
    'kpt_110', 'kpt_111', 'kpt_112', 'kpt_113', 'kpt_114', 'kpt_115', 'kpt_116', 'kpt_117', 'kpt_118', 'kpt_119',
    'kpt_120', 'kpt_121', 'kpt_122', 'kpt_123', 'kpt_124', 'kpt_125', 'kpt_126', 'kpt_127', 'kpt_128', 'kpt_129',
    'kpt_130', 'kpt_131', 'kpt_132'
]
OFFICIAL_MMPOSE_COCO_WHOLEBODY_SKELETON_LINKS = [
    [15, 13], [13, 11], [16, 14], [14, 12], [11, 12], [5, 11], [6, 12],
    [5, 6], [5, 7], [6, 8], [7, 9], [8, 10], [1, 2], [0, 1], [0, 2],
    [1, 3], [2, 4], [3, 5], [4, 6], [17, 18], [18, 19], [19, 20],
    [20, 21], [21, 22], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28],
    [28, 29], [23, 30], [30, 31], [31, 32], [32, 33], [23, 34], [34, 35],
    [35, 36], [36, 37], [23, 38], [38, 39], [39, 40], [40, 41], [42, 43],
    [43, 44], [44, 45], [45, 46], [42, 47], [47, 48], [48, 49], [49, 50],
    [42, 51], [51, 52], [52, 53], [53, 54], [42, 55], [55, 56], [56, 57],
    [57, 58], [59, 60], [60, 61], [61, 62], [62, 63], [59, 64], [64, 65],
    [65, 66], [66, 67], [59, 68], [68, 69], [69, 70], [70, 71], [59, 72],
    [72, 73], [73, 74], [74, 75], [59, 76], [76, 77], [77, 78], [78, 79],
    [80, 81], [81, 82], [82, 83], [80, 84], [84, 85], [85, 86], [80, 87],
    [87, 88], [88, 89], [80, 90], [91, 92], [92, 93], [93, 94], [94, 95],
    [91, 96], [96, 97], [97, 98], [98, 99], [91, 100], [100, 101],
    [101, 102], [102, 103], [91, 104], [104, 105], [105, 106],
    [106, 107], [91, 108], [108, 109], [109, 110], [110, 111], [112, 113],
    [113, 114], [114, 115], [115, 116], [112, 117], [117, 118],
    [118, 119], [119, 120], [112, 121], [121, 122], [122, 123],
    [123, 124], [112, 125], [125, 126], [126, 127], [127, 128],
    [112, 129], [129, 130], [130, 131], [131, 132]
]

# --- Initialize models ---
# Initialize detector within the 'mmdet' scope
print("Initializing detector...")
with DefaultScope.overwrite_default_scope(scope_name='mmdet'): # CORRECTED: Using the classmethod context manager
    detector = init_detector(local_det_config_file, local_det_checkpoint_file, device=device)
print("Detector initialized.")

# Initialize pose estimator within the 'mmpose' scope
print("Initializing pose estimator...")
with DefaultScope.overwrite_default_scope(scope_name='mmpose'): # CORRECTED: Using the classmethod context manager
    pose_estimator = init_pose_estimator(local_pose_config_file, local_pose_checkpoint_file, device=device)
    
    # --- Manually set the dataset_meta for COCO-WholeBody ---
    print("Explicitly setting COCO-WholeBody dataset_meta...")
    
    new_dataset_meta = {
        'keypoint_names': COCO_WHOLEBODY_KEYPOINT_NAMES,
        'num_keypoints': len(COCO_WHOLEBODY_KEYPOINT_NAMES),
        'skeleton_links': OFFICIAL_MMPOSE_COCO_WHOLEBODY_SKELETON_LINKS,
    }
    
    if hasattr(pose_estimator, 'dataset_meta') and pose_estimator.dataset_meta is not None:
        pose_estimator.dataset_meta.update(new_dataset_meta)
        print("Updated existing pose_estimator.dataset_meta.")
    else:
        pose_estimator.dataset_meta = new_dataset_meta
        print("Set new pose_estimator.dataset_meta.")
            
    print(f"Using {len(pose_estimator.dataset_meta['keypoint_names'])} keypoint names.")
    print(f"Using {len(pose_estimator.dataset_meta['skeleton_links'])} skeleton links.")

print("Pose estimator initialized.")
print("Models loaded successfully.")

In [ ]:
from mmengine.registry import DefaultScope # Ensure DefaultScope is imported
import numpy as np # Ensure numpy is imported
import torch # Ensure torch is imported for type checking

def extract_pose_from_frame(frame_bgr, detector_model, pose_estimator_model, person_label_id=0, detection_threshold=0.5):
    """
    Extracts whole-body pose from a single frame for one person.
    Manages MMLab scopes using DefaultScope.overwrite_default_scope context manager.
    Handles cases where pred_instances data might already be numpy arrays.
    """
    
    person_bboxes_np = None
    combined_pose_data = None
    
    # --- Part 1: Object Detection (MMDetection) ---
    with DefaultScope.overwrite_default_scope(scope_name='mmdet'):
        det_results = inference_detector(detector_model, frame_bgr)
        pred_instances = det_results.pred_instances
        
        person_indices = (pred_instances.labels == person_label_id) & (pred_instances.scores > detection_threshold)
        person_bboxes_data = pred_instances.bboxes[person_indices] # Could be tensor or numpy

        if len(person_bboxes_data) == 0:
            return None 

        areas = (person_bboxes_data[:, 2] - person_bboxes_data[:, 0]) * (person_bboxes_data[:, 3] - person_bboxes_data[:, 1])
        
        # areas might be a tensor, ensure it's on CPU for argmax if so, then convert to numpy if needed for indexing
        if isinstance(areas, torch.Tensor):
            largest_person_idx = areas.cpu().numpy().argmax()
        else: # Assuming numpy array
            largest_person_idx = areas.argmax()
            
        main_person_bbox_data = person_bboxes_data[largest_person_idx:largest_person_idx+1]
        
        if len(main_person_bbox_data) == 0: # Should be main_person_bbox_data.shape[0] for numpy
            return None

        if isinstance(main_person_bbox_data, torch.Tensor):
            person_bboxes_np = main_person_bbox_data.cpu().numpy()
        else: # Assuming numpy array
            person_bboxes_np = main_person_bbox_data


    if person_bboxes_np is None or person_bboxes_np.shape[0] == 0:
        return None

    # --- Part 2: Pose Estimation (MMPose) ---
    with DefaultScope.overwrite_default_scope(scope_name='mmpose'):
        pose_results = inference_topdown(pose_estimator_model, frame_bgr, person_bboxes_np)
        
        if not pose_results:
            return None 
            
        data_sample = pose_results[0] # This is a PoseDataSample object
        
        # Access keypoints and scores from the PoseDataSample's pred_instances
        # These might be Tensors or NumPy arrays depending on the exact version and internal processing
        
        keypoints_data = data_sample.pred_instances.keypoints[0] # Get the data for the first (and only) detected person
        keypoint_scores_data = data_sample.pred_instances.keypoint_scores[0]

        # Convert to NumPy array if they are PyTorch Tensors
        if isinstance(keypoints_data, torch.Tensor):
            keypoints_np = keypoints_data.cpu().numpy()
        elif isinstance(keypoints_data, np.ndarray):
            keypoints_np = keypoints_data
        else:
            print(f"Warning: Unexpected type for keypoints_data: {type(keypoints_data)}")
            return None # Or handle error appropriately

        if isinstance(keypoint_scores_data, torch.Tensor):
            keypoint_scores_np = keypoint_scores_data.cpu().numpy()
        elif isinstance(keypoint_scores_data, np.ndarray):
            keypoint_scores_np = keypoint_scores_data
        else:
            print(f"Warning: Unexpected type for keypoint_scores_data: {type(keypoint_scores_data)}")
            return None # Or handle error appropriately
        
        combined_pose_data = np.concatenate((keypoints_np, keypoint_scores_np[:, np.newaxis]), axis=1)
    
    return combined_pose_data

print("Pose extraction function defined (handles numpy/tensor for keypoints).")

In [ ]:
EXCEL_FILE_PATH = '/kaggle/input/aslvid/asllvd_signs_2024_06_27.xlsx'
ASLLVD_BASE_URL = "http://csr.bu.edu/ftp/asl/asllvd/asl-data2/quicktime/"

# Directories for downloaded and processed files
BASE_WORKING_DIR = Path('/kaggle/working/')
FULL_VIDEO_DOWNLOAD_DIR = BASE_WORKING_DIR / 'full_source_videos'
SEGMENTED_VIDEO_DIR = BASE_WORKING_DIR / 'segmented_sign_clips'
POSE_DATA_OUTPUT_DIR = BASE_WORKING_DIR / 'pose_data'

FULL_VIDEO_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
SEGMENTED_VIDEO_DIR.mkdir(parents=True, exist_ok=True)
POSE_DATA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def load_excel_data(excel_path, num_samples=None):
    """Loads data from the Excel file. Optionally selects a number of samples."""
    try:
        df = pd.read_excel(excel_path)
        print(f"Successfully loaded Excel file. Found {len(df)} rows.")
        # Basic cleaning: remove rows where essential data might be missing
        essential_cols = ['full video file', 
                          'start frame of the sign (relative to full videos)', 
                          'end frame of the sign (relative to full videos)',
                          'Video ID number',
                          'occurrence label']
        df.dropna(subset=essential_cols, inplace=True)
        print(f"After dropping rows with NA in essential columns, {len(df)} rows remaining.")

        # Ensure frame numbers are integers
        df['start frame of the sign (relative to full videos)'] = df['start frame of the sign (relative to full videos)'].astype(int)
        df['end frame of the sign (relative to full videos)'] = df['end frame of the sign (relative to full videos)'].astype(int)

        if num_samples is not None and num_samples < len(df):
            print(f"Selecting {num_samples} random samples.")
            return df.sample(n=num_samples, random_state=42) # Use a random_state for reproducibility
        return df
    except FileNotFoundError:
        print(f"Error: Excel file not found at {excel_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error loading or processing Excel file: {e}")
        return pd.DataFrame()

def parse_full_video_filename(full_video_name_from_excel):
    """
    Parses 'ASL_YYYY_MM_DD_scene<SCENE_NUM>-camera<CAM_NUM>.mov'
    Returns (session_dir_name, scene_num_str, camera_num_str, actual_filename_on_server)
    Example: ASL_2008_01_11_scene71-camera1.mov
    -> ("ASL_2008_01_11", "71", "1", "scene71-camera1.mov")
    """
    match = re.match(r'(ASL_\d{4}_\d{2}_\d{2})_scene(\d+)-camera(\d+)\.mov', full_video_name_from_excel)
    if match:
        session_dir = match.group(1)
        scene_num = match.group(2)
        camera_num = match.group(3)
        actual_filename = f"scene{scene_num}-camera{camera_num}.mov"
        return session_dir, scene_num, camera_num, actual_filename
    else:
        print(f"Warning: Could not parse filename format: {full_video_name_from_excel}")
        return None, None, None, None

def download_full_video(video_info_tuple, download_dir):
    """Downloads the full video if it doesn't already exist."""
    session_dir_name, scene_num, camera_num, actual_filename = video_info_tuple
    if not all(video_info_tuple): # Check if parsing failed
        return None

    # Construct the part of the URL specific to this video
    # e.g. ASL_2008_01_11/scene71-camera1.mov
    video_url_path = f"{session_dir_name}/{actual_filename}"
    full_url = ASLLVD_BASE_URL + video_url_path
    
    local_video_path = Path(download_dir) / actual_filename
    
    if local_video_path.exists():
        print(f"Full video already exists: {local_video_path}")
        return local_video_path

    print(f"Downloading full video from {full_url} to {local_video_path}...")
    try:
        response = requests.get(full_url, stream=True, timeout=300) # 5 min timeout
        response.raise_for_status()
        with open(local_video_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192 * 4): # 32KB chunks
                f.write(chunk)
        print(f"Download complete: {local_video_path}")
        return local_video_path
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {full_url}: {e}")
        if local_video_path.exists():
            local_video_path.unlink() # Remove partial download
        return None

def segment_video_ffmpeg(full_video_path, output_segment_path, start_frame, end_frame):
    """Segments a video using ffmpeg. Frames are 1-based and inclusive."""
    # ffmpeg's trim filter: end_frame is exclusive, so add 1 if our end_frame is inclusive
    # The filter expects frame numbers.
    command = [
        'ffmpeg',
        '-i', str(full_video_path),
        '-vf', f'trim=start_frame={start_frame-1}:end_frame={end_frame},setpts=PTS-STARTPTS', # trim frames are 0-indexed
        '-an',  # No audio
        '-y',   # Overwrite output files
        str(output_segment_path)
    ]
    print(f"Running ffmpeg command: {' '.join(command)}")
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        print(f"ffmpeg stdout: {result.stdout}")
        print(f"ffmpeg stderr: {result.stderr}") # ffmpeg often prints info to stderr
        if output_segment_path.exists() and output_segment_path.stat().st_size > 0:
            print(f"Successfully segmented video to {output_segment_path}")
            return output_segment_path
        else:
            print(f"Error: Segmented video not created or is empty: {output_segment_path}")
            return None
    except subprocess.CalledProcessError as e:
        print(f"Error during ffmpeg segmentation for {full_video_path}:")
        print(f"Command: {' '.join(e.cmd)}")
        print(f"Return code: {e.returncode}")
        print(f"Stdout: {e.stdout}")
        print(f"Stderr: {e.stderr}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during ffmpeg segmentation: {e}")
        return None

print("Utility functions for data loading, download, and segmentation defined.")
# Test parser
test_fn = "ASL_2008_01_11_scene71-camera1.mov"
parsed_info = parse_full_video_filename(test_fn)
print(f"Parsed info for {test_fn}: {parsed_info}")
if parsed_info and all(parsed_info):
    session, scene, cam, actual_fn = parsed_info
    test_url = ASLLVD_BASE_URL + f"{session}/{actual_fn}"
    print(f"Constructed test URL: {test_url}")

test_fn_2 = "ASL_2008_03_13_scene111-camera2.mov" # Example for camera2
parsed_info_2 = parse_full_video_filename(test_fn_2)
print(f"Parsed info for {test_fn_2}: {parsed_info_2}")
if parsed_info_2 and all(parsed_info_2):
    session, scene, cam, actual_fn = parsed_info_2
    test_url_2 = ASLLVD_BASE_URL + f"{session}/{actual_fn}"
    print(f"Constructed test URL for camera2: {test_url_2}")

In [ ]:
NUM_SIGNS_TO_PROCESS = 3  # Start with a small number of signs
CLEANUP_TEMP_FILES = False # Set to False to keep downloaded/segmented videos for inspection

excel_data_df = load_excel_data(EXCEL_FILE_PATH, num_samples=NUM_SIGNS_TO_PROCESS)

if excel_data_df.empty:
    print("No data loaded from Excel. Exiting processing loop.")
else:
    print(f"Processing {len(excel_data_df)} selected sign entries...")
    
    # Ensure correct column names from your Excel file
    # From your image: 'Video ID number', 'occurrence label', 
    # 'start frame of the sign (relative to full videos)', 'end frame of the sign (relative to full videos)',
    # 'full video file'
    col_video_id = 'Video ID number'
    col_occurrence_label = 'occurrence label'
    col_start_frame_sign = 'start frame of the sign (relative to full videos)'
    col_end_frame_sign = 'end frame of the sign (relative to full videos)'
    col_full_video_file = 'full video file'

    for index, row in tqdm(excel_data_df.iterrows(), total=excel_data_df.shape[0], desc="Processing Signs"):
        video_id = str(row[col_video_id])
        occurrence_label = str(row[col_occurrence_label]).replace('/', '_').replace('\\', '_').replace(' ', '') # Sanitize for filename
        start_frame = int(row[col_start_frame_sign])
        end_frame = int(row[col_end_frame_sign])
        full_video_filename_excel = row[col_full_video_file]

        print(f"\nProcessing sign: ID {video_id}, Label {occurrence_label}, Source {full_video_filename_excel}, Frames {start_frame}-{end_frame}")

        # 1. Parse filename and get download info
        video_info_tuple = parse_full_video_filename(full_video_filename_excel)
        if not all(video_info_tuple):
            print(f"Skipping due to filename parse error: {full_video_filename_excel}")
            continue
        
        # 2. Download full source video (if not already present)
        downloaded_full_video_path = download_full_video(video_info_tuple, FULL_VIDEO_DOWNLOAD_DIR)
        if not downloaded_full_video_path:
            print(f"Skipping due to download error for: {full_video_filename_excel}")
            continue

        # 3. Segment the sign clip
        # Sanitize occurrence_label further for use in filenames
        safe_occurrence_label = re.sub(r'[^a-zA-Z0-9_+-]', '', occurrence_label)
        segmented_clip_name = f"sign_{video_id}_{safe_occurrence_label}_{start_frame}-{end_frame}.mp4"
        temp_segmented_clip_path = SEGMENTED_VIDEO_DIR / segmented_clip_name
        
        final_segmented_clip_path = segment_video_ffmpeg(downloaded_full_video_path, temp_segmented_clip_path, start_frame, end_frame)
        if not final_segmented_clip_path:
            print(f"Skipping due to segmentation error for sign {video_id} from {full_video_filename_excel}")
            continue
            
        # 4. Perform Pose Estimation on the segmented clip
        output_npz_filename = f"poses_{video_id}_{safe_occurrence_label}_{start_frame}-{end_frame}.npz"
        output_npz_path = POSE_DATA_OUTPUT_DIR / output_npz_filename

        if output_npz_path.exists():
            print(f"Pose data already exists, skipping: {output_npz_path}")
            if CLEANUP_TEMP_FILES and final_segmented_clip_path.exists():
                final_segmented_clip_path.unlink() # Clean up segmented clip
            continue

        cap = cv2.VideoCapture(str(final_segmented_clip_path))
        if not cap.isOpened():
            print(f"Error: Could not open segmented video {final_segmented_clip_path}")
            if CLEANUP_TEMP_FILES and final_segmented_clip_path.exists():
                final_segmented_clip_path.unlink()
            continue

        all_frame_poses = []
        frame_idx = 0
        total_clip_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        pbar_frames = tqdm(total=total_clip_frames, desc=f"Frames in {segmented_clip_name}", leave=False)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            pose_data = extract_pose_from_frame(frame, detector, pose_estimator)
            if pose_data is not None:
                all_frame_poses.append(pose_data)
            else:
                num_keypoints = pose_estimator.cfg.data_cfg.num_keypoints if hasattr(pose_estimator.cfg, 'data_cfg') else 133
                all_frame_poses.append(np.full((num_keypoints, 3), np.nan))
            
            frame_idx += 1
            pbar_frames.update(1)
        
        cap.release()
        pbar_frames.close()

        if all_frame_poses:
            all_frame_poses_np = np.array(all_frame_poses)
            np.savez_compressed(output_npz_path, poses=all_frame_poses_np)
            print(f"Saved pose data to {output_npz_path} (Shape: {all_frame_poses_np.shape})")
        else:
            print(f"No poses extracted for segmented clip {final_segmented_clip_path}")

        # 5. Cleanup temporary segmented clip
        if CLEANUP_TEMP_FILES and final_segmented_clip_path.exists():
            print(f"Cleaning up segmented clip: {final_segmented_clip_path}")
            final_segmented_clip_path.unlink()
            
    # Optional: Cleanup downloaded full videos after all processing if desired
    # if CLEANUP_TEMP_FILES:
    #     for item in FULL_VIDEO_DOWNLOAD_DIR.iterdir():
    #         if item.is_file(): item.unlink()
    #     FULL_VIDEO_DOWNLOAD_DIR.rmdir() # if empty

print("\nAll selected signs processed.")

In [ ]:
NUM_SIGNS_TO_PROCESS = 20  # Increase to build a larger dictionary, e.g., 20, 50, 100
CLEANUP_TEMP_FILES = False # Keep segmented videos for now for easier visual reference

excel_data_df = load_excel_data(EXCEL_FILE_PATH, num_samples=NUM_SIGNS_TO_PROCESS)

# --- GLOSS TO NPZ MAPPING ---
gloss_to_npz_map = {}
GLOSS_MAP_FILE = BASE_WORKING_DIR / 'gloss_to_pose_map.json' # Store as JSON

if excel_data_df.empty:
    print("No data loaded from Excel. Exiting processing loop.")
else:
    print(f"Processing {len(excel_data_df)} selected sign entries...")
    
    col_video_id = 'Video ID number'
    # Choose the best column for the primary gloss. 
    # 'occurrence label' might have variants. 'entry/variant gloss label' or 'Class Label' might be better.
    # Let's assume 'Class Label' is a good candidate for a cleaned gloss.
    col_gloss_label = 'Class Label' # Or 'occurrence label', 'entry/variant gloss label'
    col_start_frame_sign = 'start frame of the sign (relative to full videos)'
    col_end_frame_sign = 'end frame of the sign (relative to full videos)'
    col_full_video_file = 'full video file'

    # Ensure the chosen gloss column exists
    if col_gloss_label not in excel_data_df.columns:
        print(f"ERROR: Chosen gloss column '{col_gloss_label}' not found in Excel. Available columns: {excel_data_df.columns.tolist()}")
        # Fallback or raise error
        # For now, let's try 'occurrence label' if 'Class Label' is missing
        if 'occurrence label' in excel_data_df.columns:
            col_gloss_label = 'occurrence label'
            print(f"Falling back to use '{col_gloss_label}' for gloss information.")
        else:
            raise KeyError(f"Neither '{col_gloss_label}' nor 'occurrence label' found in Excel columns.")


    for index, row in tqdm(excel_data_df.iterrows(), total=excel_data_df.shape[0], desc="Processing Signs"):
        video_id = str(row[col_video_id])
        
        # Extract and clean gloss
        raw_gloss = str(row[col_gloss_label])
        # Basic cleaning: uppercase, remove annotations like (1), +, #. Adapt as needed.
        # This cleaning needs to be consistent with how your T2G system will output glosses.
        cleaned_gloss = re.sub(r'\(\d+\)', '', raw_gloss).strip() # Remove (1), (2) etc.
        cleaned_gloss = cleaned_gloss.replace('+', '').replace('#', '').upper()
        # If a gloss can have multiple parts separated by '/', decide how to handle:
        # Option 1: Store each part separately if they represent distinct signs.
        # Option 2: Keep them together if it's a compound sign that has a single video.
        # For ASLLVD, often a "Gloss Variant" or "occurrence label" is one unit.
        # The example `(1)CHEAT` in 'Class Label' suggests it's often a single unit.
        # If your T2G produces "CHEAT", you want this to match.
        
        start_frame = int(row[col_start_frame_sign])
        end_frame = int(row[col_end_frame_sign])
        full_video_filename_excel = row[col_full_video_file]

        print(f"\nProcessing sign: ID {video_id}, Raw Gloss '{raw_gloss}' -> Cleaned Gloss '{cleaned_gloss}', Source {full_video_filename_excel}, Frames {start_frame}-{end_frame}")

        video_info_tuple = parse_full_video_filename(full_video_filename_excel)
        if not all(video_info_tuple):
            print(f"Skipping due to filename parse error: {full_video_filename_excel}")
            continue
        
        downloaded_full_video_path = download_full_video(video_info_tuple, FULL_VIDEO_DOWNLOAD_DIR)
        if not downloaded_full_video_path:
            print(f"Skipping due to download error for: {full_video_filename_excel}")
            continue

        # Use cleaned_gloss for a more stable filename component if it's simple enough
        # Otherwise, stick to video_id and occurrence_label for uniqueness.
        # Let's use video_id and a safe version of the raw_gloss/occurrence for filename parts
        # to ensure uniqueness if multiple different signs have the same cleaned_gloss (e.g. homonyms if not distinguished by context)
        # For the dictionary, we use `cleaned_gloss`.
        
        # Filename for segmented clip (using video_id and frames for uniqueness)
        # Segmented clip name:
        safe_raw_gloss_for_filename = re.sub(r'[^a-zA-Z0-9_+-]', '', raw_gloss) # Sanitize original label for filename
        segmented_clip_name = f"sign_{video_id}_{safe_raw_gloss_for_filename}_{start_frame}-{end_frame}.mp4"
        temp_segmented_clip_path = SEGMENTED_VIDEO_DIR / segmented_clip_name
        
        final_segmented_clip_path = segment_video_ffmpeg(downloaded_full_video_path, temp_segmented_clip_path, start_frame, end_frame)
        if not final_segmented_clip_path:
            print(f"Skipping due to segmentation error for sign {video_id} from {full_video_filename_excel}")
            continue
            
        output_npz_filename = f"poses_{video_id}_{safe_raw_gloss_for_filename}_{start_frame}-{end_frame}.npz"
        output_npz_path = POSE_DATA_OUTPUT_DIR / output_npz_filename

        if os.path.exists(output_npz_path):
            print(f"Pose data already exists: {output_npz_path}")
        else:
            cap = cv2.VideoCapture(str(final_segmented_clip_path))
            if not cap.isOpened():
                print(f"Error: Could not open segmented video {final_segmented_clip_path}")
                if CLEANUP_TEMP_FILES and final_segmented_clip_path.exists():
                    final_segmented_clip_path.unlink()
                continue

            all_frame_poses = []
            frame_idx = 0
            total_clip_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            pbar_frames = tqdm(total=total_clip_frames, desc=f"Frames in {segmented_clip_name}", leave=False)
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                pose_data = extract_pose_from_frame(frame, detector, pose_estimator)
                if pose_data is not None:
                    all_frame_poses.append(pose_data)
                else:
                    num_keypoints = 133 # Default based on RTMPose wholebody
                    # Try to get from config if possible, though pose_estimator might not be fully initialized with data_cfg here
                    try: num_keypoints = pose_estimator.cfg.data_cfg.num_keypoints
                    except: pass
                    all_frame_poses.append(np.full((num_keypoints, 3), np.nan))
                
                frame_idx += 1
                pbar_frames.update(1)
            
            cap.release()
            pbar_frames.close()

            if all_frame_poses:
                all_frame_poses_np = np.array(all_frame_poses)
                np.savez_compressed(output_npz_path, poses=all_frame_poses_np)
                print(f"Saved pose data to {output_npz_path} (Shape: {all_frame_poses_np.shape})")
            else:
                print(f"No poses extracted for segmented clip {final_segmented_clip_path}")
        
        # --- Add to gloss_to_npz_map ---
        # Only add if NPZ was created or already existed
        if os.path.exists(output_npz_path):
            if cleaned_gloss not in gloss_to_npz_map:
                gloss_to_npz_map[cleaned_gloss] = []
            # Store the relative path for portability if this map is used elsewhere
            relative_npz_path = str(output_npz_path.relative_to(BASE_WORKING_DIR))
            if relative_npz_path not in gloss_to_npz_map[cleaned_gloss]:
                 gloss_to_npz_map[cleaned_gloss].append(relative_npz_path)
            print(f"Mapped gloss '{cleaned_gloss}' to NPZ: {relative_npz_path}")
        else:
            print(f"Skipping mapping for gloss '{cleaned_gloss}' as NPZ file was not created: {output_npz_path}")


        if CLEANUP_TEMP_FILES and final_segmented_clip_path.exists() and not os.path.exists(output_npz_path):
            # Clean up segmented clip only if NPZ was NOT created and we intend to clean up
            print(f"Cleaning up failed segmented clip: {final_segmented_clip_path}")
            final_segmented_clip_path.unlink()
            
# --- Save the gloss_to_npz_map ---
import json
with open(GLOSS_MAP_FILE, 'w') as f:
    json.dump(gloss_to_npz_map, f, indent=4)
print(f"\nGloss to NPZ map saved to: {GLOSS_MAP_FILE}")
print(f"Total unique glosses mapped: {len(gloss_to_npz_map)}")

# Optional: Cleanup downloaded full videos
# if CLEANUP_TEMP_FILES:
# for item in FULL_VIDEO_DOWNLOAD_DIR.iterdir():
# if item.is_file(): item.unlink()
# if FULL_VIDEO_DOWNLOAD_DIR.exists() and not any(FULL_VIDEO_DOWNLOAD_DIR.iterdir()):
# FULL_VIDEO_DOWNLOAD_DIR.rmdir()

print("\nAll selected signs processed and mapped.")

In [ ]:
# This cell is to verify the output.
print(f"\n--- Inspecting Output in {POSE_DATA_OUTPUT_DIR} ---")
generated_files = [f for f in os.listdir(POSE_DATA_OUTPUT_DIR) if f.endswith('.npz')]

if generated_files:
    print(f"Found {len(generated_files)} NPZ files.")
    # Load the first generated .npz file as an example
    example_npz_path = POSE_DATA_OUTPUT_DIR / generated_files[0]
    print(f"Loading example NPZ file: {example_npz_path}")
    
    try:
        data = np.load(example_npz_path)
        print("Keys in the NPZ file:", data.files) 
        
        if 'poses' in data:
            poses_array = data['poses']
            print(f"Shape of the 'poses' array: {poses_array.shape}")
            
            if poses_array.shape[0] > 0 and poses_array.shape[1] > 0:
                print(f"Example pose data (first keypoint, first frame): {poses_array[0, 0, :]}")
            else:
                print("Pose array is empty or has an unexpected shape.")
        else:
            print("The key 'poses' was not found in the NPZ file.")
        data.close() # Important to close the file
    except Exception as e:
        print(f"Error loading or inspecting NPZ file {example_npz_path}: {e}")
        
else:
    print(f"No .npz files found in {POSE_DATA_OUTPUT_DIR}. Ensure the previous cell ran correctly and processed signs.")


In [ ]:
import cv2
import numpy as np
import os
import torch # For PoseDataSample if needed by visualizer
from pathlib import Path
from tqdm import tqdm

# --- Configuration for Visualization ---
# Pick one of the processed signs to visualize by its NPZ file name
# Example: poses_5939_MOST_3242-3272.npz
EXAMPLE_NPZ_FILENAME = "poses_5939_MOST_3242-3272.npz" # Change this to an existing NPZ file

# Derive corresponding segmented video filename (adjust logic if your naming differs)
# Assumes segmented video was like: sign_VIDEOID_LABEL_START-END.mp4
base_name_from_npz = EXAMPLE_NPZ_FILENAME.replace('poses_', 'sign_').replace('.npz', '.mp4')
SEGMENTED_CLIP_TO_VISUALIZE = SEGMENTED_VIDEO_DIR / base_name_from_npz # Path defined in Cell 6

POSE_DATA_NPZ_PATH = POSE_DATA_OUTPUT_DIR / EXAMPLE_NPZ_FILENAME # Path defined in Cell 6
VISUALIZED_OUTPUT_DIR = BASE_WORKING_DIR / 'visualized_output'
VISUALIZED_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
VISUALIZED_VIDEO_PATH = VISUALIZED_OUTPUT_DIR / f"vis_{base_name_from_npz}"

# --- Initialize MMPose Visualizer ---
# The pose_estimator (from Cell 4) holds the visualizer and its dataset_meta
# We need to ensure the scope is correct for the visualizer if it uses registry components
if 'pose_estimator' not in locals() or 'detector' not in locals():
    print("Please ensure Cell 4 has been run to initialize pose_estimator and detector.")
else:
    try:
        from mmengine.registry import DefaultScope
        from mmpose.visualization import PoseLocalVisualizer # MMPose v1.x typically uses this

        # It's safer to re-initialize a visualizer instance here if needed,
        # or ensure the one in pose_estimator is correctly configured.
        # For MMPose 1.x, PoseLocalVisualizer is common.
        
        visualizer = PoseLocalVisualizer(
            name='visualizer_for_output', # name for the visualizer instance
            # is_openset=True, # May not be needed, depends on visualizer version
        )
        visualizer.set_dataset_meta(
            dataset_meta=pose_estimator.dataset_meta, 
            skeleton_style='coco_wholebody' # Or 'openpose' etc. based on your model
        )
        
        # You can customize drawing style
        # visualizer.radius = 3
        # visualizer.line_width = 2
        # visualizer.alpha = 0.7 # For semi-transparent overlays

        print(f"Attempting to visualize: {SEGMENTED_CLIP_TO_VISUALIZE}")
        print(f"Using pose data from: {POSE_DATA_NPZ_PATH}")

        if not SEGMENTED_CLIP_TO_VISUALIZE.exists():
            print(f"ERROR: Segmented video not found: {SEGMENTED_CLIP_TO_VISUALIZE}")
            print("Ensure 'CLEANUP_TEMP_FILES = False' in Cell 7 and re-run it, or that the file exists.")
        elif not POSE_DATA_NPZ_PATH.exists():
            print(f"ERROR: Pose NPZ file not found: {POSE_DATA_NPZ_PATH}")
        else:
            # Load pose data
            pose_data_archive = np.load(POSE_DATA_NPZ_PATH)
            all_frame_poses_np = pose_data_archive['poses'] # Shape: (num_frames, num_keypoints, 3)

            # Open video capture
            cap = cv2.VideoCapture(str(SEGMENTED_CLIP_TO_VISUALIZE))
            if not cap.isOpened():
                print(f"Error: Could not open video {SEGMENTED_CLIP_TO_VISUALIZE}")
            else:
                fps = cap.get(cv2.CAP_PROP_FPS)
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                
                # Define video writer
                fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                video_writer = cv2.VideoWriter(str(VISUALIZED_VIDEO_PATH), fourcc, fps, (width, height))
                print(f"Outputting visualized video to: {VISUALIZED_VIDEO_PATH}")

                num_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                num_pose_frames = all_frame_poses_np.shape[0]

                if num_video_frames != num_pose_frames:
                    print(f"Warning: Mismatch in frame count! Video has {num_video_frames}, Poses have {num_pose_frames}.")
                    print("Visualization will proceed up to the shorter duration.")
                
                frames_to_process = min(num_video_frames, num_pose_frames)

                for frame_idx in tqdm(range(frames_to_process), desc="Visualizing frames"):
                    ret, frame_bgr = cap.read()
                    if not ret:
                        break

                    current_pose_keypoints = all_frame_poses_np[frame_idx, :, :2] # (N, K, 2) -> x,y
                    current_pose_scores = all_frame_poses_np[frame_idx, :, 2]    # (N, K)   -> scores
                    
                    # Create a PoseDataSample-like structure for the visualizer
                    # For MMPose 1.x, visualizer.add_datasample expects a data_sample object
                    # which has `pred_instances` with `keypoints` and `keypoint_scores`.
                    # We only have one instance (person) per frame from our saved data.
                    
                    # Create a dictionary to mimic PoseDataSample structure for visualization
                    # This structure might need adjustment based on the specific PoseLocalVisualizer version
                    drawn_frame = visualizer.draw_instance_pred(
                        image=frame_bgr.copy(), # Draw on a copy
                        instances={ # Mimicking structure, may need specific keys
                            'keypoints': current_pose_keypoints[np.newaxis, ...], # Add batch dim: (1, K, 2)
                            'keypoint_scores': current_pose_scores[np.newaxis, ...] # Add batch dim: (1, K)
                        }
                    )
                    
                    video_writer.write(drawn_frame)
                
                cap.release()
                video_writer.release()
                print(f"Finished visualizing. Video saved to {VISUALIZED_VIDEO_PATH}")
                # You can then download this video from the Kaggle output directory.

    except Exception as e:
        print(f"An error occurred during visualization setup or processing: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
import cv2
import numpy as np
import os
import torch # For PoseDataSample if needed by visualizer
from pathlib import Path
from tqdm import tqdm

# --- Configuration for Visualization ---
# Ensure this NPZ file exists from your Cell 7 run
EXAMPLE_NPZ_FILENAME = "poses_5939_MOST_3242-3272.npz" # Change this if needed
# Or pick another one from your output:
# EXAMPLE_NPZ_FILENAME = "poses_615_RUN_2780-2845.npz" 

# Derive corresponding segmented video filename
base_name_from_npz = EXAMPLE_NPZ_FILENAME.replace('poses_', 'sign_').replace('.npz', '.mp4')
SEGMENTED_CLIP_TO_VISUALIZE = SEGMENTED_VIDEO_DIR / base_name_from_npz
POSE_DATA_NPZ_PATH = POSE_DATA_OUTPUT_DIR / EXAMPLE_NPZ_FILENAME
VISUALIZED_OUTPUT_DIR = BASE_WORKING_DIR / 'visualized_output'
VISUALIZED_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
VISUALIZED_VIDEO_PATH = VISUALIZED_OUTPUT_DIR / f"vis_{base_name_from_npz}"

# Check if models are loaded (pose_estimator should have dataset_meta)
if 'pose_estimator' not in locals():
    print("ERROR: 'pose_estimator' not found. Please ensure Cell 4 (model initialization) has been run successfully.")
else:
    try:
        from mmengine.registry import DefaultScope
        from mmpose.visualization import PoseLocalVisualizer 
        from mmpose.structures import PoseDataSample # For creating the data sample structure
        from mmengine.structures import InstanceData # For pred_instances

        # Initialize visualizer (or reuse from pose_estimator if configured)
        # For consistency and explicit control, let's create a new one for this task.
        visualizer = PoseLocalVisualizer(
            name='visualizer_for_output_video',
            radius=3,  # Radius of a keypoint
            line_width=2,  # Line width of a skeleton
            # is_openset=True, # May not be needed for drawing known skeletons
        )
        visualizer.set_dataset_meta(
            dataset_meta=pose_estimator.dataset_meta, 
            # Ensure the skeleton style matches your model.
            # RTMPose-wholebody often uses 'coco_wholebody' or 'openpose' depending on the exact variant.
            # Check pose_estimator.dataset_meta['skeleton_links'] or similar if unsure.
            skeleton_style='coco_wholebody' 
        )
        
        print(f"Attempting to visualize: {SEGMENTED_CLIP_TO_VISUALIZE}")
        print(f"Using pose data from: {POSE_DATA_NPZ_PATH}")

        if not SEGMENTED_CLIP_TO_VISUALIZE.exists():
            print(f"ERROR: Segmented video not found: {SEGMENTED_CLIP_TO_VISUALIZE}")
            print("Ensure 'CLEANUP_TEMP_FILES = False' in Cell 7 and re-run it, or that the file exists.")
        elif not POSE_DATA_NPZ_PATH.exists():
            print(f"ERROR: Pose NPZ file not found: {POSE_DATA_NPZ_PATH}")
        else:
            pose_data_archive = np.load(POSE_DATA_NPZ_PATH)
            all_frame_poses_np = pose_data_archive['poses'] 

            cap = cv2.VideoCapture(str(SEGMENTED_CLIP_TO_VISUALIZE))
            if not cap.isOpened():
                print(f"Error: Could not open video {SEGMENTED_CLIP_TO_VISUALIZE}")
            else:
                fps = cap.get(cv2.CAP_PROP_FPS)
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                
                fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                video_writer = cv2.VideoWriter(str(VISUALIZED_VIDEO_PATH), fourcc, fps, (width, height))
                print(f"Outputting visualized video to: {VISUALIZED_VIDEO_PATH}")

                num_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                num_pose_frames = all_frame_poses_np.shape[0]

                if num_video_frames != num_pose_frames:
                    print(f"Warning: Mismatch in frame count! Video has {num_video_frames}, Poses have {num_pose_frames}.")
                
                frames_to_process = min(num_video_frames, num_pose_frames)

                for frame_idx in tqdm(range(frames_to_process), desc="Visualizing frames"):
                    ret, frame_bgr = cap.read()
                    if not ret:
                        break

                    current_pose_keypoints = all_frame_poses_np[frame_idx, :, :2] # (K, 2) -> x,y
                    current_pose_scores = all_frame_poses_np[frame_idx, :, 2]    # (K)   -> scores
                    
                    # Create a PoseDataSample structure for the visualizer
                    data_sample = PoseDataSample()
                    pred_instances = InstanceData()
                    pred_instances.keypoints = current_pose_keypoints[np.newaxis, ...] # Shape: (1, K, 2)
                    pred_instances.keypoint_scores = current_pose_scores[np.newaxis, ...] # Shape: (1, K)
                    data_sample.pred_instances = pred_instances
                    
                    # Add metainfo if required by visualizer (e.g., image path for some visualizers, not always needed for drawing)
                    # data_sample.set_metainfo({'img_path': 'dummy_path_for_visualization'})


                    # Use add_datasample and get_image
                    # The visualizer draws on an internal canvas or the provided image.
                    # We provide the current frame_bgr to draw upon.
                    with DefaultScope.overwrite_default_scope(scope_name='mmpose'): # Ensure correct scope for visualizer internals
                        visualizer.add_datasample(
                            name='frame_visualization', # A name for this drawing operation
                            image=frame_bgr.copy(), # Draw on a copy of the current frame
                            data_sample=data_sample,
                            draw_gt=False, # We don't have ground truth here
                            draw_heatmap=False,
                            draw_bbox=False, # We are providing keypoints directly
                            # show=False, # Don't show interactively
                            # wait_time=0,
                            # out_file=None # We are getting the image to write to video
                        )
                        drawn_frame = visualizer.get_image()
                    
                    video_writer.write(drawn_frame)
                
                cap.release()
                video_writer.release()
                print(f"Finished visualizing. Video saved to {VISUALIZED_VIDEO_PATH}")

    except Exception as e:
        print(f"An error occurred during visualization setup or processing: {e}")
        import traceback
        traceback.print_exc()


In [ ]:
    # Temporary Cell (e.g., Cell 4.1) to inspect skeleton_links

    if 'pose_estimator' in locals():
        print("--- pose_estimator.dataset_meta ---")
        # Print the whole dataset_meta to see what's available
        # print(pose_estimator.dataset_meta) 
        
        print("\n--- Keypoint Names (first 20 for brevity) ---")
        if 'keypoint_names' in pose_estimator.dataset_meta:
            print(pose_estimator.dataset_meta['keypoint_names'][:20])
        else:
            print("'keypoint_names' not found in dataset_meta.")

        print("\n--- Skeleton Links ---")
        if 'skeleton_links' in pose_estimator.dataset_meta:
            actual_skeleton_links = pose_estimator.dataset_meta['skeleton_links']
            print(f"Found {len(actual_skeleton_links)} skeleton links.")
            print("First 128 links:", actual_skeleton_links[:128])
            # You can assign this to a variable to copy-paste into Cell 10
            # For example:
            # my_links_for_cell_10 = actual_skeleton_links 
        else:
            print("'skeleton_links' not found in dataset_meta. This is unexpected for visualization.")
            print("Available keys in dataset_meta:", pose_estimator.dataset_meta.keys())

    else:
        print("Variable 'pose_estimator' is not defined. Please run Cell 4 first.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML # For displaying the animation in Jupyter/Kaggle
from pathlib import Path
import os # For os.listdir

# --- Configuration ---
# Ensure EXAMPLE_NPZ_FILENAME is defined from Cell 9 or set it here
if 'EXAMPLE_NPZ_FILENAME' not in locals() or not EXAMPLE_NPZ_FILENAME:
    print("Warning: EXAMPLE_NPZ_FILENAME not set, picking one from output if available.")
    npz_files_in_output = [f for f in os.listdir(POSE_DATA_OUTPUT_DIR) if f.endswith('.npz')]
    if npz_files_in_output:
        EXAMPLE_NPZ_FILENAME = npz_files_in_output[0]
        print(f"Using NPZ file for animation: {EXAMPLE_NPZ_FILENAME}")
    else:
        print("ERROR: No NPZ files found in output directory. Cannot create animation.")
        raise FileNotFoundError("No NPZ files available for animation.")

NPZ_TO_ANIMATE = POSE_DATA_OUTPUT_DIR / EXAMPLE_NPZ_FILENAME
ANIMATION_OUTPUT_DIR = BASE_WORKING_DIR / 'animations'
ANIMATION_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
ANIMATION_FILENAME = ANIMATION_OUTPUT_DIR / f"anim_{NPZ_TO_ANIMATE.stem}.mp4"

# --- Load Pose Data ---
if not NPZ_TO_ANIMATE.exists():
    print(f"ERROR: NPZ file not found: {NPZ_TO_ANIMATE}")
else:
    print(f"Loading pose data from: {NPZ_TO_ANIMATE}")
    pose_data_archive = np.load(NPZ_TO_ANIMATE)
    poses_array = pose_data_archive['poses']
    num_frames, num_keypoints_in_data, _ = poses_array.shape
    print(f"Loaded {num_frames} frames with {num_keypoints_in_data} keypoints each.")

    # --- Define Skeleton Connections (Using the 128 links from your dataset_meta) ---
    actual_skeleton_links = [
        [15, 13], [13, 11], [16, 14], [14, 12], [11, 12], [5, 11], [6, 12], 
        [5, 6], [5, 7], [6, 8], [7, 9], [8, 10], [1, 2], [0, 1], [0, 2], 
        [1, 3], [2, 4], [3, 5], [4, 6], [17, 18], [18, 19], [19, 20], 
        [20, 21], [21, 22], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], 
        [28, 29], [23, 30], [30, 31], [31, 32], [32, 33], [23, 34], [34, 35], 
        [35, 36], [36, 37], [23, 38], [38, 39], [39, 40], [40, 41], [42, 43], 
        [43, 44], [44, 45], [45, 46], [42, 47], [47, 48], [48, 49], [49, 50], 
        [42, 51], [51, 52], [52, 53], [53, 54], [42, 55], [55, 56], [56, 57], 
        [57, 58], [59, 60], [60, 61], [61, 62], [62, 63], [59, 64], [64, 65], 
        [65, 66], [66, 67], [59, 68], [68, 69], [69, 70], [70, 71], [59, 72], 
        [72, 73], [73, 74], [74, 75], [59, 76], [76, 77], [77, 78], [78, 79], 
        [80, 81], [81, 82], [82, 83], [80, 84], [84, 85], [85, 86], [80, 87], 
        [87, 88], [88, 89], [80, 90], [91, 92], [92, 93], [93, 94], [94, 95], 
        [91, 96], [96, 97], [97, 98], [98, 99], [91, 100], [100, 101], 
        [101, 102], [102, 103], [91, 104], [104, 105], [105, 106], [106, 107], 
        [91, 108], [108, 109], [109, 110], [110, 111], [112, 113], 
        [113, 114], [114, 115], [115, 116], [112, 117], [117, 118], 
        [118, 119], [119, 120], [112, 121], [121, 122], [122, 123], [123, 124], 
        [112, 125], [125, 126], [126, 127], [127, 128], [112, 129], 
        [129, 130], [130, 131], [131, 132]
    ]
    print(f"Using {len(actual_skeleton_links)} skeleton links for animation.")

    fig, ax = plt.subplots(figsize=(8, 6))
    lines = [ax.plot([], [], 'o-', color='cyan', markersize=1, linewidth=0.75)[0] for _ in actual_skeleton_links] 
    
    valid_poses = poses_array[~np.isnan(poses_array).any(axis=(1,2))]
    if valid_poses.shape[0] > 0:
        all_x = valid_poses[:, :, 0].flatten()
        all_y = valid_poses[:, :, 1].flatten()
        all_x_no_nan = all_x[~np.isnan(all_x)]
        all_y_no_nan = all_y[~np.isnan(all_y)]

        if all_x_no_nan.size > 0 and all_y_no_nan.size > 0:
            padding = 50 
            x_min, x_max = np.min(all_x_no_nan) - padding, np.max(all_x_no_nan) + padding
            y_min, y_max = np.min(all_y_no_nan) - padding, np.max(all_y_no_nan) + padding
            ax.set_xlim(x_min, x_max)
            ax.set_ylim(y_max, y_min) 
        else:
            print("Warning: No valid (non-NaN) keypoints found to determine plot limits. Using defaults.")
            ax.set_xlim(0, 640) 
            ax.set_ylim(480, 0)
    else:
        print("Warning: No frames with valid poses found to determine plot limits. Using defaults.")
        ax.set_xlim(0, 640) 
        ax.set_ylim(480, 0)
        
    ax.set_aspect('equal', adjustable='box')
    ax.set_facecolor('black') 
    fig.patch.set_facecolor('black') 
    ax.tick_params(axis='x', colors='white') 
    ax.tick_params(axis='y', colors='white') 
    ax.spines['bottom'].set_color('white') 
    ax.spines['top'].set_color('white')   
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    plt.title(f"2D Pose Animation: {NPZ_TO_ANIMATE.name}", color='white')

    confidence_threshold = 0.3 

    def init_animation():
        for line in lines:
            line.set_data([], [])
        return lines

    def update_animation(frame_idx):
        keypoints_frame = poses_array[frame_idx, :, :2] 
        scores_frame = poses_array[frame_idx, :, 2]     

        for i, (idx1, idx2) in enumerate(actual_skeleton_links):
            if idx1 < num_keypoints_in_data and idx2 < num_keypoints_in_data:
                is_valid_p1 = not np.isnan(keypoints_frame[idx1, 0]) and not np.isnan(keypoints_frame[idx1, 1])
                is_valid_p2 = not np.isnan(keypoints_frame[idx2, 0]) and not np.isnan(keypoints_frame[idx2, 1])

                if is_valid_p1 and is_valid_p2 and \
                   scores_frame[idx1] > confidence_threshold and \
                   scores_frame[idx2] > confidence_threshold:
                    x_coords = [keypoints_frame[idx1, 0], keypoints_frame[idx2, 0]]
                    y_coords = [keypoints_frame[idx1, 1], keypoints_frame[idx2, 1]]
                    lines[i].set_data(x_coords, y_coords)
                    lines[i].set_visible(True)
                else:
                    lines[i].set_data([], []) 
                    lines[i].set_visible(False) 
            else:
                 lines[i].set_data([], []) 
                 lines[i].set_visible(False) 
        return lines

    ani_fps = 30 
    ani = animation.FuncAnimation(fig, update_animation, frames=num_frames,
                                  init_func=init_animation, blit=True, interval=int(1000/ani_fps))

    print(f"Saving animation to {ANIMATION_FILENAME}...")
    try:
        ani.save(ANIMATION_FILENAME, writer='ffmpeg', fps=ani_fps, dpi=150, 
                 progress_callback=lambda current_frame, total_frames: print(f"Saving frame {current_frame+1}/{total_frames}", end='\r') if total_frames and (current_frame + 1) % 10 == 0 or current_frame + 1 == total_frames else None)
        print("\nAnimation saved.")
        plt.close(fig) 
        # print("Preparing HTML for display (this might take a moment)...")
        # display_html = HTML(ani.to_jshtml(fps=ani_fps))
        # display(display_html)
        # plt.close(fig)
        print(f"To view, download: {ANIMATION_FILENAME} from the 'animations' directory in your Kaggle output.")
    except Exception as e:
        print(f"\nError saving animation: {e}")
        import traceback
        traceback.print_exc()
        plt.close(fig)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import os
from IPython.display import HTML, display
import json

# --- Configuration ---
POSE_DATA_DIR = "/kaggle/working/pose_data/"
OUTPUT_ANIM_DIR = "/kaggle/working/animated_skeletons/"
GLOSS_TO_POSE_MAP_FILE = "/kaggle/working/gloss_to_pose_map.json"
KEYPOINT_CONFIDENCE_THRESHOLD = 0.3

# Ensure output directory exists
os.makedirs(OUTPUT_ANIM_DIR, exist_ok=True)

# Load the gloss_to_pose_map
try:
    with open(GLOSS_TO_POSE_MAP_FILE, 'r') as f:
        gloss_to_pose_map = json.load(f)
    print(f"Successfully loaded gloss_to_pose_map from {GLOSS_TO_POSE_MAP_FILE}")
    NUM_ANIMATIONS_TO_GENERATE = len(gloss_to_pose_map) # Process all loaded glosses
    print(f"Set to generate {NUM_ANIMATIONS_TO_GENERATE} animations.")
except FileNotFoundError:
    print(f"ERROR: Could not find {GLOSS_TO_POSE_MAP_FILE}. Make sure Cell 7 ran correctly.")
    gloss_to_pose_map = {}
    NUM_ANIMATIONS_TO_GENERATE = 0
except json.JSONDecodeError:
    print(f"ERROR: Could not decode {GLOSS_TO_POSE_MAP_FILE}. Check its content.")
    gloss_to_pose_map = {}
    NUM_ANIMATIONS_TO_GENERATE = 0


# Get skeleton links
if 'pose_estimator' in globals() and hasattr(pose_estimator, 'dataset_meta') and 'skeleton_links' in pose_estimator.dataset_meta:
    actual_skeleton_links = pose_estimator.dataset_meta['skeleton_links']
    print(f"Using {len(actual_skeleton_links)} skeleton links from pose_estimator.dataset_meta.")
else:
    print("Warning: pose_estimator or its dataset_meta not found. Using a default COCO-WholeBody skeleton definition.")
    COCO_WHOLEBODY_SKELETON_LINKS = [
        [15, 13], [13, 11], [16, 14], [14, 12], [11, 12], [5, 11], [6, 12], [5, 6], [5, 7], [6, 8], [7, 9], [8, 10],
        [1, 2], [0, 1], [0, 2], [1, 3], [2, 4], [3, 5], [4, 6], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22],
        [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [23, 30], [30, 31], [31, 32], [32, 33], [23, 34],
        [34, 35], [35, 36], [36, 37], [23, 38], [38, 39], [39, 40], [40, 41], [42, 43], [43, 44], [44, 45], [45, 46],
        [42, 47], [47, 48], [48, 49], [49, 50], [42, 51], [51, 52], [52, 53], [53, 54], [42, 55], [55, 56], [56, 57],
        [57, 58], [59, 60], [60, 61], [61, 62], [62, 63], [59, 64], [64, 65], [65, 66], [66, 67], [59, 68], [68, 69],
        [69, 70], [70, 71], [59, 72], [72, 73], [73, 74], [74, 75], [59, 76], [76, 77], [77, 78], [78, 79], [80, 81],
        [81, 82], [82, 83], [80, 84], [84, 85], [85, 86], [80, 87], [87, 88], [88, 89], [80, 90], [91, 92], [92, 93],
        [93, 94], [94, 95], [91, 96], [96, 97], [97, 98], [98, 99], [91, 100], [100, 101], [101, 102], [102, 103],
        [91, 104], [104, 105], [105, 106], [106, 107], [91, 108], [108, 109], [109, 110], [110, 111], [112, 113],
        [113, 114], [114, 115], [115, 116], [112, 117], [117, 118], [118, 119], [119, 120], [112, 121], [121, 122],
        [122, 123], [123, 124], [112, 125], [125, 126], [126, 127], [127, 128], [112, 129], [129, 130], [130, 131],
        [131, 132]]
    actual_skeleton_links = COCO_WHOLEBODY_SKELETON_LINKS
    print(f"Using a default COCO-WholeBody skeleton definition with {len(actual_skeleton_links)} links.")

processed_glosses_count = 0
for gloss_idx, (gloss, path_value) in enumerate(gloss_to_pose_map.items()):
    if processed_glosses_count >= NUM_ANIMATIONS_TO_GENERATE:
        break
    
    current_npz_path_str = None
    if isinstance(path_value, list):
        if len(path_value) > 0 and isinstance(path_value[0], str):
            current_npz_path_str = path_value[0]
        else:
            print(f"ERROR: Gloss '{gloss}' has a list value, but it's empty or doesn't contain a string. Value: {path_value}. Skipping.")
            continue
    elif isinstance(path_value, str):
        current_npz_path_str = path_value
    else:
        print(f"ERROR: Path value for gloss '{gloss}' is not a string or list of strings. Type: {type(path_value)}. Value: {path_value}. Skipping.")
        continue

    if not current_npz_path_str:
        print(f"ERROR: Could not determine NPZ path for gloss '{gloss}'. Skipping.")
        continue

    if "pose_data/" in current_npz_path_str:
         base_working_dir = "/kaggle/working/"
         npz_file_path = os.path.join(base_working_dir, current_npz_path_str)
    else:
         npz_file_path = os.path.join(POSE_DATA_DIR, current_npz_path_str)
    
    if not os.path.exists(npz_file_path):
        alternative_path = os.path.join(POSE_DATA_DIR, os.path.basename(current_npz_path_str))
        if os.path.exists(alternative_path):
            npz_file_path = alternative_path
        else:
            print(f"Could not find NPZ file for gloss '{gloss}'. Checked: '{npz_file_path}' and '{alternative_path}'. Skipping.")
            continue

    try:
        all_poses_npz = np.load(npz_file_path)
    except Exception as e:
        print(f"Error loading NPZ file {npz_file_path}: {e}. Skipping.")
        continue

    if 'poses' not in all_poses_npz:
        print(f"Key 'poses' not found in {npz_file_path}. Skipping.")
        continue

    animation_poses = all_poses_npz['poses']
    num_frames, num_keypoints, _ = animation_poses.shape

    if num_frames == 0 or num_keypoints == 0:
        print(f"No frames or keypoints in {npz_file_path} for gloss '{gloss}'. Skipping.")
        continue

    print(f"\nGenerating animation for gloss: '{gloss}' (Frames: {num_frames}, Keypoints: {num_keypoints}) from {npz_file_path}")

    fig, ax = plt.subplots()
    
    # Filter poses by confidence for calculating bounds
    confident_poses = animation_poses[animation_poses[:, :, 2] > KEYPOINT_CONFIDENCE_THRESHOLD]
    
    if confident_poses.shape[0] == 0: # Check if any confident keypoints exist across all frames
        print(f"No keypoints above confidence threshold for gloss '{gloss}' across all frames. Skipping animation.")
        plt.close(fig)
        continue

    all_x_coords = confident_poses[:, 0]
    all_y_coords = confident_poses[:, 1]
    
    global_min_x, global_max_x = np.min(all_x_coords), np.max(all_x_coords)
    global_min_y, global_max_y = np.min(all_y_coords), np.max(all_y_coords)
    padding = 20 
    
    # DEBUG: Print axis limits
    print(f"  Axis limits for '{gloss}': X=({global_min_x - padding:.2f}, {global_max_x + padding:.2f}), Y=({global_min_y - padding:.2f}, {global_max_y + padding:.2f})")

    # --- DEBUG: Save first frame as static image ---
    # (Only for the very first animation being processed for quicker debugging)
    save_static_frame_debug = (processed_glosses_count == 0) 


    def update(i):
        ax.cla() # Clear axis
        frame_poses = animation_poses[i]
        
        ax.set_xlim(global_min_x - padding, global_max_x + padding)
        ax.set_ylim(global_min_y - padding, global_max_y + padding)
        ax.set_aspect('equal', adjustable='box')
        ax.set_title(f"Gloss: {gloss} - Frame {i+1}/{num_frames}")
        ax.invert_yaxis()

        # --- DEBUG: Print some keypoint data for the first frame of the first animation ---
        if i == 0 and save_static_frame_debug:
            print(f"    Frame 0 Keypoints (first 5 for '{gloss}'):")
            for kp_idx_print in range(min(5, num_keypoints)):
                x_p, y_p, conf_p = frame_poses[kp_idx_print]
                if conf_p > KEYPOINT_CONFIDENCE_THRESHOLD:
                    print(f"      KP{kp_idx_print}: ({x_p:.2f}, {y_p:.2f}), Conf: {conf_p:.2f}")

        # Draw skeleton links
        for kp_idx1, kp_idx2 in actual_skeleton_links:
            if kp_idx1 < num_keypoints and kp_idx2 < num_keypoints: # Bounds check
                x1, y1, conf1 = frame_poses[kp_idx1]
                x2, y2, conf2 = frame_poses[kp_idx2]
                if conf1 > KEYPOINT_CONFIDENCE_THRESHOLD and conf2 > KEYPOINT_CONFIDENCE_THRESHOLD:
                    ax.plot([x1, x2], [y1, y2], color='red', linewidth=2.5, zorder=1)
        
        # Draw keypoints (joints)
        visible_kps_x = []
        visible_kps_y = []
        for kp_idx in range(num_keypoints):
            x, y, conf = frame_poses[kp_idx]
            if conf > KEYPOINT_CONFIDENCE_THRESHOLD:
                visible_kps_x.append(x)
                visible_kps_y.append(y)
        
        if visible_kps_x: # Check if there are any keypoints to draw
            ax.scatter(visible_kps_x, visible_kps_y, c='blue', s=30, zorder=2)

        # --- DEBUG: Save first frame as static image ---
        if i == 0 and save_static_frame_debug:
            debug_frame_path = os.path.join(OUTPUT_ANIM_DIR, f"debug_frame_{gloss_idx}.png")
            try:
                plt.savefig(debug_frame_path)
                print(f"    Saved debug frame to {debug_frame_path}")
            except Exception as e_fig:
                print(f"    Error saving debug frame: {e_fig}")
        
        return [ax] # For blit=True, though we set it to False

    # Create animation, set blit=False for debugging
    ani = animation.FuncAnimation(fig, update, frames=num_frames, blit=False, interval=50) 
    
    safe_gloss_name = "".join(c if c.isalnum() else "_" for c in gloss)
    output_path = os.path.join(OUTPUT_ANIM_DIR, f"anim_{safe_gloss_name}.mp4")
    
    try:
        ani.save(output_path, writer='ffmpeg', fps=20)
        print(f"  Saved animation to {output_path}")
    except Exception as e:
        print(f"  Error saving animation for gloss '{gloss}': {e}")
    finally:
        plt.close(fig) # Ensure figure is closed

    processed_glosses_count += 1

if processed_glosses_count == 0:
    print("\nNo animations were generated. Check NPZ files, paths, and confidence thresholds.")
else:
    print(f"\nFinished generating {processed_glosses_count} animations in {OUTPUT_ANIM_DIR}")


In [ ]:
# Cell 11: 2D Sprite Character Animation with Pygame (More Complete)
import pygame
import numpy as np
import math
import os
import json
from PIL import Image # For saving pygame surface to image file
import subprocess # For running ffmpeg

# --- Configuration ---
POSE_DATA_DIR = "/kaggle/working/pose_data/"
PYGAME_ANIM_DIR = "/kaggle/working/pygame_animations/"
PYGAME_FRAMES_DIR = "/kaggle/working/pygame_frames_temp/" # Temp storage for frames for current video
GLOSS_TO_POSE_MAP_FILE = "/kaggle/working/gloss_to_pose_map.json"
KEYPOINT_CONFIDENCE_THRESHOLD = 0.3
NUM_PYGAME_ANIMATIONS_TO_GENERATE = 1 # Generate for one sign first for testing

# Colors
BG_COLOR = (255, 255, 255) # White background
LIMB_COLOR = (50, 150, 255, 200) # RGBA, A for alpha (semi-transparent)
HEAD_COLOR = (255, 200, 150, 200) # RGBA
TORSO_COLOR = (100, 100, 100, 200) # RGBA
JOINT_COLOR = (255, 0, 0) # For drawing debug joints

# Sprite dimensions (placeholders)
LIMB_WIDTH = 20     # Width of the limb/torso rectangles
HEAD_RADIUS = 25
JOINT_RADIUS = 5 # For debug drawing

# Screen/Surface padding
PADDING = 50

# Ensure output directories exist
os.makedirs(PYGAME_ANIM_DIR, exist_ok=True)
# For PYGAME_FRAMES_DIR, we'll clear it for each new video
# os.makedirs(PYGAME_FRAMES_DIR, exist_ok=True)

# --- Pygame Initialization ---
# Check if Pygame display has been initialized to avoid re-init issues in notebooks
if not pygame.display.get_init():
    os.environ['SDL_VIDEODRIVER'] = 'dummy' # Use dummy driver for headless environments
    pygame.init()
    print("Pygame initialized.")
else:
    print("Pygame already initialized.")


# --- Keypoint Indices (COCO WholeBody - 133 keypoints) ---
KP_NOSE = 0; KP_L_EYE = 1; KP_R_EYE = 2; KP_L_EAR = 3; KP_R_EAR = 4
KP_L_SHOULDER = 5; KP_R_SHOULDER = 6; KP_L_ELBOW = 7; KP_R_ELBOW = 8
KP_L_WRIST = 9; KP_R_WRIST = 10; KP_L_HIP = 11; KP_R_HIP = 12
KP_L_KNEE = 13; KP_R_KNEE = 14; KP_L_ANKLE = 15; KP_R_ANKLE = 16

# --- Helper Functions ---
def calculate_angle_degrees(p1x, p1y, p2x, p2y):
    return math.degrees(math.atan2(p2y - p1y, p2x - p1x))

def calculate_distance(p1x, p1y, p2x, p2y):
    return math.sqrt((p2x - p1x)**2 + (p2y - p1y)**2)

def draw_limb_sprite(surface, p1_data, p2_data, limb_color, limb_width):
    """Draws a limb between two keypoints p1 and p2 with specified width and color."""
    x1, y1, conf1 = p1_data
    x2, y2, conf2 = p2_data

    if conf1 < KEYPOINT_CONFIDENCE_THRESHOLD or conf2 < KEYPOINT_CONFIDENCE_THRESHOLD:
        return

    length = calculate_distance(x1, y1, x2, y2)
    if length < 1: 
        return

    limb_surface = pygame.Surface((int(length), int(limb_width)), pygame.SRCALPHA) # Ensure int dimensions
    limb_surface.fill(limb_color)

    angle = calculate_angle_degrees(x1, y1, x2, y2)
    rotated_limb_surface = pygame.transform.rotate(limb_surface, -angle)
    
    rad_angle = math.radians(angle)
    center_x = x1 + (length / 2.0) * math.cos(rad_angle)
    center_y = y1 + (length / 2.0) * math.sin(rad_angle)
    
    blit_rect = rotated_limb_surface.get_rect(center=(int(center_x), int(center_y))) # Ensure int center
    surface.blit(rotated_limb_surface, blit_rect.topleft)


def draw_head_sprite(surface, nose_kp, l_shoulder_kp, r_shoulder_kp, head_color, radius):
    """Draws a head, positioned relative to nose and shoulders."""
    nx, ny, nconf = nose_kp
    lsx, lsy, lsconf = l_shoulder_kp
    rsx, rsy, rsconf = r_shoulder_kp

    if nconf < KEYPOINT_CONFIDENCE_THRESHOLD :
        return

    head_center_x = nx
    head_center_y = ny - radius * 0.5 
    
    if lsconf > KEYPOINT_CONFIDENCE_THRESHOLD and rsconf > KEYPOINT_CONFIDENCE_THRESHOLD:
        shoulder_mid_y = (lsy + rsy) / 2
        if ny > shoulder_mid_y :
             head_center_y = shoulder_mid_y - radius 

    pygame.draw.circle(surface, head_color, (int(head_center_x), int(head_center_y)), int(radius))


def draw_torso_sprite(surface, sl_kp, sr_kp, hl_kp, hr_kp, torso_color):
    """Draws a torso as a polygon given shoulder and hip keypoints."""
    points_data = [sl_kp, sr_kp, hr_kp, hl_kp] 
    polygon_points = []
    for p_data in points_data:
        if p_data[2] > KEYPOINT_CONFIDENCE_THRESHOLD:
            x_coord = int(p_data[0]) # Ensure integer coordinates
            y_coord = int(p_data[1]) # Ensure integer coordinates
            polygon_points.append((x_coord, y_coord))
        else: 
            return 
            
    if len(polygon_points) == 4: 
        try:
            pygame.draw.polygon(surface, torso_color, polygon_points)
        except TypeError as e:
            print(f"ERROR in pygame.draw.polygon: {e}")
            print(f"  Torso color: {torso_color}, type: {type(torso_color)}")
            print(f"  Polygon points: {polygon_points}")
            for i_pt, pt_val in enumerate(polygon_points):
                print(f"    Point {i_pt}: {pt_val}, type x: {type(pt_val[0])}, type y: {type(pt_val[1])}")
            # raise # Optionally re-raise after debugging
    # else:
    #     print(f"DEBUG: Torso not drawn, {len(polygon_points)} valid points.")


# --- Load Data ---
try:
    with open(GLOSS_TO_POSE_MAP_FILE, 'r') as f:
        gloss_to_pose_map = json.load(f)
    print(f"Successfully loaded gloss_to_pose_map from {GLOSS_TO_POSE_MAP_FILE}")
except Exception as e:
    print(f"Error loading gloss_to_pose_map.json: {e}")
    gloss_to_pose_map = {}

if not gloss_to_pose_map:
    print("Gloss map is empty. Cannot proceed.")
else:
    processed_animations_count = 0
    for gloss, path_values in gloss_to_pose_map.items():
        if processed_animations_count >= NUM_PYGAME_ANIMATIONS_TO_GENERATE:
            break

        print(f"\nProcessing gloss: {gloss}")
        
        current_frames_dir = os.path.join(PYGAME_FRAMES_DIR, gloss.replace('/', '_')) # Subdirectory for each gloss
        if os.path.exists(current_frames_dir):
            for f_name in os.listdir(current_frames_dir):
                os.remove(os.path.join(current_frames_dir, f_name))
        else:
            os.makedirs(current_frames_dir)

        current_npz_path_str = None
        if isinstance(path_values, list) and path_values:
            current_npz_path_str = path_values[0]
        elif isinstance(path_values, str):
            current_npz_path_str = path_values
        else:
            print(f"  Invalid path value for gloss '{gloss}': {path_values}. Skipping.")
            continue

        if "pose_data/" in current_npz_path_str:
             npz_file_path = os.path.join("/kaggle/working/", current_npz_path_str)
        else:
             npz_file_path = os.path.join(POSE_DATA_DIR, current_npz_path_str)

        if not os.path.exists(npz_file_path):
            print(f"  NPZ file not found: {npz_file_path}. Skipping.")
            continue
        
        try:
            animation_poses_all_data = np.load(npz_file_path)
            animation_poses = animation_poses_all_data['poses'] 
        except Exception as e:
            print(f"  Error loading NPZ {npz_file_path}: {e}. Skipping.")
            continue

        num_frames, num_keypoints, _ = animation_poses.shape
        if num_frames == 0:
            print(f"  No frames in {npz_file_path}. Skipping.")
            continue
        
        confident_coords_x = []
        confident_coords_y = []
        for frame_idx in range(num_frames):
            for kp_idx in range(num_keypoints):
                if animation_poses[frame_idx, kp_idx, 2] > KEYPOINT_CONFIDENCE_THRESHOLD:
                    confident_coords_x.append(animation_poses[frame_idx, kp_idx, 0])
                    confident_coords_y.append(animation_poses[frame_idx, kp_idx, 1])
        
        if not confident_coords_x or not confident_coords_y:
            print(f"  No confident keypoints found for gloss '{gloss}' to determine bounds. Skipping.")
            continue

        min_x, max_x = min(confident_coords_x), max(confident_coords_x)
        min_y, max_y = min(confident_coords_y), max(confident_coords_y)

        surface_w_raw = int(max_x - min_x + 2 * PADDING)
        surface_h_raw = int(max_y - min_y + 2 * PADDING)
        
        surface_w = surface_w_raw + (surface_w_raw % 2) 
        surface_h = surface_h_raw + (surface_h_raw % 2) 
        
        offset_x = -min_x + PADDING
        offset_y = -min_y + PADDING

        print(f"  Raw Surface size for '{gloss}': {surface_w_raw}x{surface_h_raw}")
        print(f"  Adjusted Even Surface size for '{gloss}': {surface_w}x{surface_h}")
        game_surface = pygame.Surface((surface_w, surface_h))

        for frame_idx in range(num_frames):
            game_surface.fill(BG_COLOR) 
            
            current_frame_poses_orig = animation_poses[frame_idx]
            current_frame_poses_transformed = np.copy(current_frame_poses_orig) # Use a different name
            for kp_idx in range(num_keypoints):
                current_frame_poses_transformed[kp_idx, 0] = current_frame_poses_orig[kp_idx, 0] + offset_x
                current_frame_poses_transformed[kp_idx, 1] = current_frame_poses_orig[kp_idx, 1] + offset_y
            
            kps = {i: tuple(current_frame_poses_transformed[i]) for i in range(num_keypoints)}

            if all(kp in kps for kp in [KP_L_SHOULDER, KP_R_SHOULDER, KP_L_HIP, KP_R_HIP]):
                draw_torso_sprite(game_surface, kps[KP_L_SHOULDER], kps[KP_R_SHOULDER], 
                                  kps[KP_L_HIP], kps[KP_R_HIP], TORSO_COLOR)

            limb_definitions = [
                (KP_L_SHOULDER, KP_L_ELBOW), (KP_L_ELBOW, KP_L_WRIST), # Left Arm
                (KP_R_SHOULDER, KP_R_ELBOW), (KP_R_ELBOW, KP_R_WRIST), # Right Arm
                (KP_L_HIP, KP_L_KNEE), (KP_L_KNEE, KP_L_ANKLE),       # Left Leg
                (KP_R_HIP, KP_R_KNEE), (KP_R_KNEE, KP_R_ANKLE)        # Right Leg
            ]
            for kp_start_idx, kp_end_idx in limb_definitions:
                if kp_start_idx in kps and kp_end_idx in kps:
                    draw_limb_sprite(game_surface, kps[kp_start_idx], kps[kp_end_idx], LIMB_COLOR, LIMB_WIDTH)
            
            if all(kp in kps for kp in [KP_NOSE, KP_L_SHOULDER, KP_R_SHOULDER]):
                 draw_head_sprite(game_surface, kps[KP_NOSE], kps[KP_L_SHOULDER], kps[KP_R_SHOULDER], HEAD_COLOR, HEAD_RADIUS)
            
            try:
                frame_pil_image = Image.frombytes('RGB', (surface_w, surface_h), pygame.image.tostring(game_surface, 'RGB'))
                frame_pil_image.save(os.path.join(current_frames_dir, f"frame_{frame_idx:05d}.png"))
            except Exception as e_save:
                print(f"    Error saving frame {frame_idx} for {gloss}: {e_save}")
                break 
        
        output_video_path = os.path.join(PYGAME_ANIM_DIR, f"pygame_anim_{gloss.replace('/', '_')}.mp4")
        ffmpeg_command = [
            'ffmpeg', '-y', '-framerate', '20', 
            '-i', os.path.join(current_frames_dir, 'frame_%05d.png'),
            '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-crf', '23', 
            output_video_path
        ]
        print(f"  Attempting to compile video: {' '.join(ffmpeg_command)}")
        try:
            result = subprocess.run(ffmpeg_command, check=False, capture_output=True, text=True) # check=False
            if result.returncode == 0:
                print(f"  Successfully compiled video to {output_video_path}")
            else:
                print(f"  ffmpeg compilation FAILED for {gloss}. Return code: {result.returncode}")
                print(f"    ffmpeg stdout:\n{result.stdout}")
                print(f"    ffmpeg stderr:\n{result.stderr}")

        except FileNotFoundError:
            print("  ERROR: ffmpeg command not found. Please ensure ffmpeg is installed and in your PATH.")
        except Exception as e_ffmpeg_general:
            print(f"  An unexpected error occurred during ffmpeg execution for {gloss}: {e_ffmpeg_general}")
            
        processed_animations_count += 1

    print(f"\nFinished generating {processed_animations_count} Pygame sprite animations in {PYGAME_ANIM_DIR}")

if not gloss_to_pose_map:
    print("No glosses found to process.")


In [ ]:
# Cell 11: 2D Sprite Character Animation with Pygame (Skeletal Hands/Face)
import pygame
import numpy as np
import math
import os
import json
from PIL import Image
import subprocess

# --- Configuration ---
POSE_DATA_DIR = "/kaggle/working/pose_data/"
PYGAME_ANIM_DIR = "/kaggle/working/pygame_animations/"
PYGAME_FRAMES_DIR_BASE = "/kaggle/working/pygame_frames_temp/" # Base for temp frames
GLOSS_TO_POSE_MAP_FILE = "/kaggle/working/gloss_to_pose_map.json"
KEYPOINT_CONFIDENCE_THRESHOLD = 0.3
NUM_PYGAME_ANIMATIONS_TO_GENERATE = 1

# Colors
BG_COLOR = (255, 255, 255)
LIMB_COLOR = (50, 150, 255, 200)
HEAD_COLOR_SHAPE = (255, 200, 150, 180) # Semi-transparent head circle
TORSO_COLOR = (100, 100, 100, 200)
HAND_SKEL_COLOR = (0, 200, 0) # Green for hand skeleton
FACE_SKEL_COLOR = (200, 0, 200) # Purple for face skeleton
JOINT_DEBUG_COLOR = (255,0,0)

# Sprite/Shape Dimensions
LIMB_WIDTH = 20
HEAD_RADIUS = 25
SKELETON_LINE_WIDTH = 3 # For hands and face
KEYPOINT_DOT_RADIUS = 2 # For drawing hand/face keypoints

PADDING = 50

# --- Pygame Initialization ---
if not pygame.display.get_init():
    os.environ['SDL_VIDEODRIVER'] = 'dummy'
    pygame.init()
    print("Pygame initialized.")
else:
    print("Pygame already initialized.")

# --- Keypoint Indices (COCO WholeBody - 133 keypoints) ---
KP_NOSE = 0; KP_L_EYE = 1; KP_R_EYE = 2; KP_L_EAR = 3; KP_R_EAR = 4
KP_L_SHOULDER = 5; KP_R_SHOULDER = 6; KP_L_ELBOW = 7; KP_R_ELBOW = 8
KP_L_WRIST = 9; KP_R_WRIST = 10; KP_L_HIP = 11; KP_R_HIP = 12
KP_L_KNEE = 13; KP_R_KNEE = 14; KP_L_ANKLE = 15; KP_R_ANKLE = 16

# Facial landmarks (subset from COCO-WholeBody points 23-90 for a more detailed face if needed)
# For now, we'll stick to the basic ones + a simple mouth if points are available
# Example mouth points (these are specific to COCO-WholeBody's 133 points)
KP_MOUTH_L_CORNER = 78 # (Original index in 133 point model)
KP_MOUTH_R_CORNER = 84
KP_MOUTH_UPPER_LIP_TOP = 81
KP_MOUTH_LOWER_LIP_BOTTOM = 87

# Hand keypoint base indices
KP_L_HAND_BASE = 91 # Left hand points are 91-111
KP_R_HAND_BASE = 112 # Right hand points are 112-132

# Hand skeleton links (local indices, 0-20 for one hand)
# 0:Wrist, 1:ThumbCMC, 2:ThumbMCP, 3:ThumbIP, 4:ThumbTIP
# 5:IndexMCP, 6:IndexPIP, 7:IndexDIP, 8:IndexTIP
# ... and so on for Middle, Ring, Pinky
HAND_SKELETON_LINKS = [
    (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),  # Index
    (0, 9), (9, 10), (10, 11), (11, 12), # Middle
    (0, 13), (13, 14), (14, 15), (15, 16), # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky
]
# Connections from MCPs to each other for palm outline (optional)
# HAND_PALM_LINKS = [(5,9), (9,13), (13,17)]


# --- Helper Functions ---
def calculate_angle_degrees(p1x, p1y, p2x, p2y):
    return math.degrees(math.atan2(p2y - p1y, p2x - p1x))

def calculate_distance(p1x, p1y, p2x, p2y):
    return math.sqrt((p2x - p1x)**2 + (p2y - p1y)**2)

def draw_limb_sprite(surface, p1_data, p2_data, limb_color, limb_width):
    x1, y1, conf1 = p1_data; x2, y2, conf2 = p2_data
    if conf1 < KEYPOINT_CONFIDENCE_THRESHOLD or conf2 < KEYPOINT_CONFIDENCE_THRESHOLD: return
    length = calculate_distance(x1, y1, x2, y2)
    if length < 1: return
    limb_surface = pygame.Surface((int(length), int(limb_width)), pygame.SRCALPHA)
    limb_surface.fill(limb_color)
    angle = calculate_angle_degrees(x1, y1, x2, y2)
    rotated_limb_surface = pygame.transform.rotate(limb_surface, -angle)
    rad_angle = math.radians(angle)
    center_x = x1 + (length / 2.0) * math.cos(rad_angle)
    center_y = y1 + (length / 2.0) * math.sin(rad_angle)
    blit_rect = rotated_limb_surface.get_rect(center=(int(center_x), int(center_y)))
    surface.blit(rotated_limb_surface, blit_rect.topleft)

def draw_head_circle(surface, nose_kp, l_shoulder_kp, r_shoulder_kp, head_color, radius):
    nx, ny, nconf = nose_kp; lsx, lsy, lsconf = l_shoulder_kp; rsx, rsy, rsconf = r_shoulder_kp
    if nconf < KEYPOINT_CONFIDENCE_THRESHOLD: return
    head_center_x = nx; head_center_y = ny - radius * 0.5
    if lsconf > KEYPOINT_CONFIDENCE_THRESHOLD and rsconf > KEYPOINT_CONFIDENCE_THRESHOLD:
        shoulder_mid_y = (lsy + rsy) / 2
        if ny > shoulder_mid_y: head_center_y = shoulder_mid_y - radius
    pygame.draw.circle(surface, head_color, (int(head_center_x), int(head_center_y)), int(radius))

def draw_torso_sprite(surface, sl_kp, sr_kp, hl_kp, hr_kp, torso_color):
    points_data = [sl_kp, sr_kp, hr_kp, hl_kp]; polygon_points = []
    for p_data in points_data:
        if p_data[2] > KEYPOINT_CONFIDENCE_THRESHOLD:
            polygon_points.append((int(p_data[0]), int(p_data[1])))
        else: return
    if len(polygon_points) == 4: pygame.draw.polygon(surface, torso_color, polygon_points)

def draw_face_features(surface, kps, color, line_width, dot_radius):
    """Draws basic facial features: eyes, nose, simple mouth line."""
    # Nose dot
    if KP_NOSE in kps and kps[KP_NOSE][2] > KEYPOINT_CONFIDENCE_THRESHOLD:
        nx, ny, _ = kps[KP_NOSE]
        pygame.draw.circle(surface, color, (int(nx), int(ny)), dot_radius)

    # Eyes
    for eye_kp_idx in [KP_L_EYE, KP_R_EYE]:
        if eye_kp_idx in kps and kps[eye_kp_idx][2] > KEYPOINT_CONFIDENCE_THRESHOLD:
            ex, ey, _ = kps[eye_kp_idx]
            pygame.draw.circle(surface, color, (int(ex), int(ey)), dot_radius + 1) # Slightly larger eyes

    # Simple mouth line (between corners, if available)
    if KP_MOUTH_L_CORNER in kps and KP_MOUTH_R_CORNER in kps:
        ml_x, ml_y, ml_conf = kps[KP_MOUTH_L_CORNER]
        mr_x, mr_y, mr_conf = kps[KP_MOUTH_R_CORNER]
        if ml_conf > KEYPOINT_CONFIDENCE_THRESHOLD and mr_conf > KEYPOINT_CONFIDENCE_THRESHOLD:
            pygame.draw.line(surface, color, (int(ml_x), int(ml_y)), (int(mr_x), int(mr_y)), line_width)
    # Or a line from nose to a chin point if mouth corners are not good. For now, just corners.


def draw_hand_skeleton(surface, kps, hand_base_idx, links, color, line_width, dot_radius):
    """Draws skeleton for one hand."""
    # Draw keypoints
    for i in range(21): # 21 keypoints per hand
        kp_idx = hand_base_idx + i
        if kp_idx in kps and kps[kp_idx][2] > KEYPOINT_CONFIDENCE_THRESHOLD:
            x, y, _ = kps[kp_idx]
            pygame.draw.circle(surface, color, (int(x), int(y)), dot_radius)

    # Draw links
    for link_start_local, link_end_local in links:
        kp_idx1 = hand_base_idx + link_start_local
        kp_idx2 = hand_base_idx + link_end_local
        if kp_idx1 in kps and kp_idx2 in kps:
            x1, y1, conf1 = kps[kp_idx1]
            x2, y2, conf2 = kps[kp_idx2]
            if conf1 > KEYPOINT_CONFIDENCE_THRESHOLD and conf2 > KEYPOINT_CONFIDENCE_THRESHOLD:
                pygame.draw.line(surface, color, (int(x1), int(y1)), (int(x2), int(y2)), line_width)

# --- Load Data ---
try:
    with open(GLOSS_TO_POSE_MAP_FILE, 'r') as f: gloss_to_pose_map = json.load(f)
    print(f"Successfully loaded gloss_to_pose_map from {GLOSS_TO_POSE_MAP_FILE}")
except Exception as e: print(f"Error loading gloss_to_pose_map.json: {e}"); gloss_to_pose_map = {}

if not gloss_to_pose_map: print("Gloss map is empty. Cannot proceed.")
else:
    processed_animations_count = 0
    for gloss, path_values in gloss_to_pose_map.items():
        if processed_animations_count >= NUM_PYGAME_ANIMATIONS_TO_GENERATE: break
        print(f"\nProcessing gloss: {gloss}")
        
        current_frames_dir = os.path.join(PYGAME_FRAMES_DIR_BASE, gloss.replace('/', '_'))
        if os.path.exists(current_frames_dir):
            for f_name in os.listdir(current_frames_dir): os.remove(os.path.join(current_frames_dir, f_name))
        else: os.makedirs(current_frames_dir)

        current_npz_path_str = path_values[0] if isinstance(path_values, list) and path_values else path_values if isinstance(path_values, str) else None
        if not current_npz_path_str: print(f"  Invalid path for '{gloss}'. Skip."); continue
        
        npz_file_path = os.path.join("/kaggle/working/", current_npz_path_str) if "pose_data/" in current_npz_path_str else os.path.join(POSE_DATA_DIR, current_npz_path_str)
        if not os.path.exists(npz_file_path): print(f"  NPZ not found: {npz_file_path}. Skip."); continue
        
        try: animation_poses = np.load(npz_file_path)['poses']
        except Exception as e: print(f"  Error loading NPZ {npz_file_path}: {e}. Skip."); continue

        num_frames, num_keypoints, _ = animation_poses.shape
        if num_frames == 0: print(f"  No frames in {npz_file_path}. Skip."); continue
        
        confident_coords_x = [animation_poses[f,k,0] for f in range(num_frames) for k in range(num_keypoints) if animation_poses[f,k,2] > KEYPOINT_CONFIDENCE_THRESHOLD]
        confident_coords_y = [animation_poses[f,k,1] for f in range(num_frames) for k in range(num_keypoints) if animation_poses[f,k,2] > KEYPOINT_CONFIDENCE_THRESHOLD]
        if not confident_coords_x: print(f"  No confident kps for '{gloss}'. Skip."); continue

        min_x,max_x=min(confident_coords_x),max(confident_coords_x); min_y,max_y=min(confident_coords_y),max(confident_coords_y)
        surface_w_raw=int(max_x-min_x+2*PADDING); surface_h_raw=int(max_y-min_y+2*PADDING)
        surface_w=surface_w_raw+(surface_w_raw%2); surface_h=surface_h_raw+(surface_h_raw%2)
        offset_x=-min_x+PADDING; offset_y=-min_y+PADDING
        print(f"  Surf size: {surface_w}x{surface_h} (Raw: {surface_w_raw}x{surface_h_raw})")
        game_surface = pygame.Surface((surface_w, surface_h))

        for frame_idx in range(num_frames):
            game_surface.fill(BG_COLOR)
            current_poses_orig = animation_poses[frame_idx]
            kps = {i: (current_poses_orig[i,0]+offset_x, current_poses_orig[i,1]+offset_y, current_poses_orig[i,2]) for i in range(num_keypoints)}

            if all(idx in kps for idx in [KP_L_SHOULDER, KP_R_SHOULDER, KP_L_HIP, KP_R_HIP]):
                draw_torso_sprite(game_surface, kps[KP_L_SHOULDER], kps[KP_R_SHOULDER], kps[KP_L_HIP], kps[KP_R_HIP], TORSO_COLOR)
            
            body_limbs = [ (KP_L_SHOULDER, KP_L_ELBOW), (KP_L_ELBOW, KP_L_WRIST), (KP_R_SHOULDER, KP_R_ELBOW), (KP_R_ELBOW, KP_R_WRIST),
                           (KP_L_HIP, KP_L_KNEE), (KP_L_KNEE, KP_L_ANKLE), (KP_R_HIP, KP_R_KNEE), (KP_R_KNEE, KP_R_ANKLE) ]
            for idx1, idx2 in body_limbs:
                if idx1 in kps and idx2 in kps: draw_limb_sprite(game_surface, kps[idx1], kps[idx2], LIMB_COLOR, LIMB_WIDTH)
            
            if all(idx in kps for idx in [KP_NOSE, KP_L_SHOULDER, KP_R_SHOULDER]): # Head circle drawn first
                draw_head_circle(game_surface, kps[KP_NOSE], kps[KP_L_SHOULDER], kps[KP_R_SHOULDER], HEAD_COLOR_SHAPE, HEAD_RADIUS)

            draw_face_features(game_surface, kps, FACE_SKEL_COLOR, SKELETON_LINE_WIDTH -1, KEYPOINT_DOT_RADIUS) # Face features on top of head circle
            draw_hand_skeleton(game_surface, kps, KP_L_HAND_BASE, HAND_SKELETON_LINKS, HAND_SKEL_COLOR, SKELETON_LINE_WIDTH, KEYPOINT_DOT_RADIUS)
            draw_hand_skeleton(game_surface, kps, KP_R_HAND_BASE, HAND_SKELETON_LINKS, HAND_SKEL_COLOR, SKELETON_LINE_WIDTH, KEYPOINT_DOT_RADIUS)

            try:
                Image.frombytes('RGB', (surface_w, surface_h), pygame.image.tostring(game_surface, 'RGB')).save(os.path.join(current_frames_dir, f"frame_{frame_idx:05d}.png"))
            except Exception as e_save: print(f"  Err save frame {frame_idx} for {gloss}: {e_save}"); break
        
        output_video_path = os.path.join(PYGAME_ANIM_DIR, f"pygame_anim_{gloss.replace('/', '_')}.mp4")
        ffmpeg_cmd = ['ffmpeg','-y','-framerate','20','-i',os.path.join(current_frames_dir,'frame_%05d.png'),'-c:v','libx264','-pix_fmt','yuv420p','-crf','23',output_video_path]
        print(f"  Compiling: {' '.join(ffmpeg_cmd)}")
        try:
            result = subprocess.run(ffmpeg_cmd, check=False, capture_output=True, text=True)
            if result.returncode == 0: print(f"  Video OK: {output_video_path}")
            else: print(f"  ffmpeg FAIL {gloss}. Code: {result.returncode}\n  stderr: {result.stderr[:1000]}")
        except Exception as e_ff: print(f"  ffmpeg EXCEPTION for {gloss}: {e_ff}")
            
        processed_animations_count += 1
    print(f"\nFinished. {processed_animations_count} Pygame animations in {PYGAME_ANIM_DIR}")

if not gloss_to_pose_map: print("No glosses found.")